In [42]:
from pathlib import Path
import pickle
import warnings

import numpy as np
import pandas as pd
from pandarallel import pandarallel
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances
from sqlalchemy import create_engine
from tqdm import tqdm

from src.models import cf, evaluate_model
from src.utilities import utilities

pandarallel.initialize()
tqdm.pandas()
warnings.filterwarnings('ignore')

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


# Load Data

In [2]:
# global variables
DATA_PATH = "data/evaluation"
CATEGORY = "Grocery_and_Gourmet_Food"
MODEL_PATH = Path(f"models/ub_cf/ub_cf_{CATEGORY}.pkl")

# reproducibility checks
SEED = 42
np.random.seed(42)

# load train dataset
train = pd.read_csv(f"{DATA_PATH}/{CATEGORY}_train.csv")

In [3]:
# checking train dataframe
train.head().append(train.tail())

,index,asin,title,categories,reviewerID,overall,reviewText,reviewTime,processedReviewText
0,0,9742356831,"Mae Ploy Green Curry Paste, 14 oz","['Grocery & Gourmet Food', 'Sauces, Gravies & ...",A23RYWDS884TUL,5.0,This curry paste makes a delicious curry. I j...,2013-05-28,curry paste delicious curry fry chicken vegeta...
1,1,9742356831,"Mae Ploy Green Curry Paste, 14 oz","['Grocery & Gourmet Food', 'Sauces, Gravies & ...",A945RBQWGZXCK,5.0,I've purchased different curries in the grocer...,2012-09-17,purchase different curry grocery store complet...
2,3,9742356831,"Mae Ploy Green Curry Paste, 14 oz","['Grocery & Gourmet Food', 'Sauces, Gravies & ...",A3AMNY44OP8AOU,4.0,I started a new diet restricting all added sug...,2014-01-23,start new diet restrict added sugar brand suga...
3,4,9742356831,"Mae Ploy Green Curry Paste, 14 oz","['Grocery & Gourmet Food', 'Sauces, Gravies & ...",A3IB4CQ2QEJLJ8,5.0,So many flavors. I can't begin to tell you how...,2014-04-27,flavor begin tell love mae ploy curry ask reci...
4,5,9742356831,"Mae Ploy Green Curry Paste, 14 oz","['Grocery & Gourmet Food', 'Sauces, Gravies & ...",AQA5DF3RWKETQ,5.0,I've used this a lot recently in some of my ch...,2012-11-27,use lot recently chicken dish use lot like spi...
47769,77420,B00I33696K,Reese's Miniature Peanut Butter Cups .31oz - 1...,"['Grocery & Gourmet Food', 'Candy & Chocolate'...",A192LQZWDYPR4U,5.0,Another quality Reese Peanut Butter Cup produc...,2014-02-27,quality reese peanut butter cup product great ...
47770,77421,B00I33696K,Reese's Miniature Peanut Butter Cups .31oz - 1...,"['Grocery & Gourmet Food', 'Candy & Chocolate'...",A2QKXW3LDQ66P5,5.0,I purchased these for my husband who has every...,2013-02-20,purchase husband love reeses valentine day pre...
47771,77430,B00ID9VSOM,"Viva Labs Organic Coconut Sugar: Non-GMO, Low-...","['Grocery & Gourmet Food', 'Cooking & Baking',...",A2P3TGJU301KXD,5.0,this stuff is INCREDIBILY yummy! SO much bette...,2014-07-15,stuff incredibily yummy good regular brown sug...
47772,77456,B00IRL93SY,Barrie House Kenya Estate - AA Single Cup Caps...,"['Grocery & Gourmet Food', 'Beverages', 'Coffe...",AEFE9VDHTQ199,5.0,"Very nice aroma, body and taste! Will buy this...",2014-05-24,nice aroma body taste buy coffee good coffee a...
47773,77508,B00ISVHJ3Y,"Wholesome Sweeteners, Organic Sweet and Lite S...","['Grocery & Gourmet Food', 'Cooking & Baking',...",A2AEZQ3DGBBLPR,2.0,This is a no go for diabetics according to my ...,2014-06-26,diabetic accord wife doctor order intention us...


In [4]:
# get user rating history
train_user_rating_history = train.groupby(["reviewerID"])["asin"].progress_apply(list)
print(train_user_rating_history)

100%|███████████████████████████████████████████| 13397/13397 [00:00<00:00, 66507.78it/s]

reviewerID
A00177463W0XWB16A9O05                             [B0029XDZIK, B0094ISOMA]
A022899328A0QROR32DCT                             [B001ACMCNU, B003TO9RSU]
A068255029AHTHDXZURNU                             [B000K8WVYA, B0094ISOMA]
A06944662TFWOKKV4GJKX                                         [B000CQBZPG]
A1004703RC79J9                                                [B001E50THY]
                                               ...                        
AZWRZZAMX90VT            [B0007R9L5Q, B000CQ01GU, B000E123IC, B000E46LZ...
AZXKAH2DE6C8A            [B000EML7DS, B000ODF2ME, B001650XUK, B0018QLG9...
AZXON596A1VXC                         [B00113SKZW, B00113ZTVK, B001L4JH5I]
AZYXC63SS008M                                                 [B0040WCQKQ]
AZZ5ASC403N74                                                 [B004U49QU2]
Name: asin, Length: 13397, dtype: object


# Generate N-Recommendations = {10, 25, 30, 45}

## Load Test Data

In [5]:
# loading test dataset
test = pd.read_csv(f"{DATA_PATH}/{CATEGORY}_test.csv")

In [6]:
test.head().append(test.tail())

,index,asin,title,categories,reviewerID,overall,reviewText,reviewTime,processedReviewText
0,2,9742356831,"Mae Ploy Green Curry Paste, 14 oz","['Grocery & Gourmet Food', 'Sauces, Gravies & ...",A1TCSC0YWT82Q0,5.0,I love ethnic foods and to cook them. I recent...,2013-08-03,love ethnic food cook recently purchase produc...
1,8,9742356831,"Mae Ploy Green Curry Paste, 14 oz","['Grocery & Gourmet Food', 'Sauces, Gravies & ...",A1Z7Y2GMAP9SRY,5.0,I like to make my own curry but this is a tast...,2014-06-27,like curry tasty alternative use base kind dif...
2,23,B00004S1C5,"Ateco Food Coloring Kit, 6 colors","['Grocery & Gourmet Food', 'Cooking & Baking',...",A14YSMLYLJEMET,1.0,This product is no where near natural / organi...,2013-03-29,product near natural organic wish review purch...
3,31,B00005344V,Traditional Medicinals Organic Breathe Easy Se...,"['Grocery & Gourmet Food', 'Beverages', 'Coffe...",A2F488C4PLWGEI,5.0,If my wife drinks a cup of this tea when she f...,2014-03-23,wife drink cup tea feel attack come help avoid...
4,32,B00005344V,Traditional Medicinals Organic Breathe Easy Se...,"['Grocery & Gourmet Food', 'Beverages', 'Coffe...",AO1HXV7DWZZIR,5.0,I don't know about the medicinal aspects of th...,2014-02-06,know medicinal aspect tea flavor downright scr...
28001,77519,B00ISVHJ3Y,"Wholesome Sweeteners, Organic Sweet and Lite S...","['Grocery & Gourmet Food', 'Cooking & Baking',...",A1WT3TVHANP7ZF,3.0,Hmmm. I really wanted to love this sweetener. ...,2014-07-22,hmmm want love sweetener half sugar half stevi...
28002,77520,B00ISVHJ3Y,"Wholesome Sweeteners, Organic Sweet and Lite S...","['Grocery & Gourmet Food', 'Cooking & Baking',...",A3NEAETOSXDBOM,5.0,"I confess I have a sweet tooth, and love the t...",2014-06-30,confess sweet tooth love taste sugar recognize...
28003,77521,B00ISVHJ3Y,"Wholesome Sweeteners, Organic Sweet and Lite S...","['Grocery & Gourmet Food', 'Cooking & Baking',...",AD1ZOPB0BBEHB,4.0,"It has a little of the stevia aftertaste, but ...",2014-07-17,little stevia aftertaste fair compromise able ...
28004,77522,B00ISVHJ3Y,"Wholesome Sweeteners, Organic Sweet and Lite S...","['Grocery & Gourmet Food', 'Cooking & Baking',...",A18ECVX2RJ7HUE,5.0,i love marinade for grilled flank steak or lon...,2014-05-30,love marinade grilled flank steak london broil...
28005,77523,B00ISVHJ3Y,"Wholesome Sweeteners, Organic Sweet and Lite S...","['Grocery & Gourmet Food', 'Cooking & Baking',...",A2G04D4QZAXL15,3.0,I've been using Truvia (a form of stevia) on m...,2014-05-27,use truvia form stevia cereal greek yogurt yea...


In [7]:
# generating test history
test_user_history = (pd.DataFrame(test.groupby(['reviewerID'])['asin']
                                  .apply(list).reset_index()))

In [8]:
print(test_user_history)

                  reviewerID  \
0      A00177463W0XWB16A9O05   
1      A022899328A0QROR32DCT   
2      A068255029AHTHDXZURNU   
3      A06944662TFWOKKV4GJKX   
4             A1004703RC79J9   
...                      ...   
13274          AZWRZZAMX90VT   
13275          AZXKAH2DE6C8A   
13276          AZXON596A1VXC   
13277          AZYXC63SS008M   
13278          AZZ5ASC403N74   

                                                    asin  
0                               [B00474OR8G, B00BFM6OAW]  
1                                           [B00CMQDKES]  
2                                           [B001FA1K2G]  
3                                           [B000GFYRHG]  
4                                           [B003GTR8IO]  
...                                                  ...  
13274  [B0007R9L4M, B000CN7BMA, B001EQ5D1K, B002VT3GX...  
13275   [B000MAK41I, B004X8TJP2, B006H34CUS, B007W14RMM]  
13276                           [B001EO5S0I, B00271QQ7Q]  
13277                    

## Instantiate Memory-based Embedding CF (User-based)

In [9]:
# instantiate model
ub_cf = cf.UserBasedCF()

In [10]:
%%time
# fit learning algorithm to training datab
ub_cf.fit(train, k_neighbours=50)

Generating user rating history...
Generating user-rating item rating (utility) matrix...
Generate user similarities matrix...
Generate k-neighbourhood of similar users...
CPU times: user 50.3 s, sys: 1.97 s, total: 52.3 s
Wall time: 32.7 s


In [11]:
# generate candidates items ranked by predicted ratings
candidate_items = ub_cf.test()

100%|██████████████████████████████████████████████| 13397/13397 [37:38<00:00,  5.93it/s]


## Save Model

In [12]:
# MODEL_PATH.parent.mkdir(parents=True, exist_ok=True)
# with open(MODEL_PATH, "wb") as f:
#     pickle.dump(ub_cf, f)

In [13]:
# # load model
# with open(MODEL_PATH, "rb") as f:
#     ub_cf = pickle.load(f)

## Loop through N = {10, 25, 30, 45}

In [14]:
# generate item popularity
item_popularity = evaluate_model.generate_item_popularity(train)

In [15]:
n_recommendations = {}
for n in [10, 25, 30, 45]:
    # retrieve the top-n items based on similarities
    # top_ns = get_top_n(candidate_items, train_user_rating_history, n)
    top_ns = ub_cf.get_top_n(n)
    # evaluate how well the recommended items predicted the future purchases
    n_recommended_items = (evaluate_model
                           .evaluate_recommendations(model_name = 'UB-CF',
                                                     top_ns = top_ns, 
                                                     user_rating_history = test_user_history,
                                                     item_popularity = item_popularity,
                                                     n = n,
                                                     mf_based = False))
    # saving the n-value and recommended items
    n_recommendations[n] = (top_ns, n_recommended_items)

The UB-CF has an average recall@10: 0.02183, average novelty@10: 0.90570
The UB-CF has an average recall@25: 0.03339, average novelty@25: 0.93888
The UB-CF has an average recall@30: 0.03621, average novelty@30: 0.94462
The UB-CF has an average recall@45: 0.04601, average novelty@45: 0.95403


# Evaluate N-Recommendations 

## N=10

In [16]:
top_ns_10 = n_recommendations[10][0]

In [17]:
utilities.retrieve_recommendations(train, top_ns_10, target_user='A5PC6KU9TQFRY')

For user: A5PC6KU9TQFRY:
Purchase History:
             asin                                              title
346    B00014HS2S  Prince of Peace Oolong Tea - 100 Tea Bags net ...
351    B00014HS2S  Prince of Peace Oolong Tea - 100 Tea Bags net ...
8980   B000H1195C  Kikkoman Egg Flower, Hot and Sour Soup, 0.88-O...
34315  B004772OWE  Celestial Seasonings Authentic Green Tea, K-Cu...
43219  B006GCMI5Q  Benson's - Table Tasty Salt Substitute - No Po...

Recommending:

         asin                                              title
0  B0042LH7TK  Green Mountain Coffee Fair Trade Pumpkin Spice...
1  B000BXSRT2     World Confections Candy Cigarettes, Pack of 24
2  B006846XQ2  Starbucks Tazo Tea * Awake * Black Tea, 16 K-C...
3  B000H11C6I  Blue Diamond Almond Nut-Thins Cracker Crisps, ...
4  B000J4IDO2  Coffee People K-Cups, Black Tiger Extra Bold, ...
5  B0029XDZIK  Keurig, The Original Donut Shop, Medium Roast ...
6  B004OINZZQ  Twinings of London Winter Spice Herbal Tea Bag...
7  B007

## N=25

In [18]:
top_ns_25 = n_recommendations[25][0]

In [19]:
utilities.retrieve_recommendations(train, top_ns_25)

For user: A2H84V55USFJQX:
Purchase History:
             asin                                              title
13844  B000ZSZ5S4  Blue Diamond Almonds, Bold Salt &amp; Vinegar,...
41088  B005K4Q1T0  Grove Square Hot Cocoa Dark Chocolate, 24 Sing...

Recommending:

          asin                                              title
0   B002HQCWYM  Twinings of London English Breakfast Tea K-Cup...
1   B004AE1FHY  Folgers Classic Roast Instant Coffee, 8 Ounce ...
2   B000EDG4V2       Bob's Red Mill Guar Gum, 8 Ounce (Case of 8)
3   B0048CHI9M  Splenda Minis Dissolvable Tablets, 100-Count (...
4   B0029XDZIK  Keurig, The Original Donut Shop, Medium Roast ...
5   B001D0IZBM                     Grocery &amp; Gourmet Food" />
6   B001NJI7GQ  Vita Coco Coconut Water, Pineapple - Naturally...
7   B004AWTR6M  Green Mountain Hot Apple Cider single serve ca...
8   B0005ZH4QI              Skippy Creamy Peanut Butter, 40 Ounce
9   B00017028M              Maldon Sea Salt Flakes, 8.5 ounce Box
10  B00

## N=30

In [20]:
top_ns_30 = n_recommendations[30][0]

In [21]:
utilities.retrieve_recommendations(train, top_ns_30)

For user: A2W9B725TZBXOX:
Purchase History:
             asin                                              title
1645   B0005Z6LLW  Ghirardelli Hot Chocolate Mix , Chocolate Moch...
4683   B000ED9L6C  Bob's Red Mill Raw Shelled Sunflower Seeds (Ke...
8508   B000GATCRQ  Dream Foods International Volcano Lemon Burst,...
15333  B0014EOUAW  V8 V-Fusion Light Peach Mango Juice Drink, 46-...
18192  B001E52ZAS  Post Shredded Wheat Lightly Frosted Cereal, Sp...
21093  B001EPQV1W  Honey Bunches of Oats with Almonds, 14.5-Ounce...
22932  B001G8UC8K  Tootsie Roll Midgees Candy 5 Pound Value Bag 7...
23025  B001GVIRD4                     Grocery &amp; Gourmet Food" />
24945  B001NC8HQS  Idahoan Buttery homestyle flavored mashed pota...
25830  B001SAQ7WQ  Progresso Traditional Soup, Chickarina (Chicke...
26956  B0025UOMY8  Maruchan Yakisoba Spicy Vegetable, 3.79-Ounce ...
33467  B00421DMLM  Libby's Splenda Sliced Peaches, 23.5-Ounce Jar...
34141  B004727CL2  Snack Factory Pretzel Crisps Everything,

## N=45

In [22]:
top_ns_45 = n_recommendations[45][0]

In [23]:
utilities.retrieve_recommendations(train, top_ns_45)

For user: A1G8ECTOF35O2R:
Purchase History:
             asin                                              title
9115   B000H26J7E  Lindt Excellence Bar, 70% Cocoa Smooth Dark Ch...
47677  B00EKLPLU4              Healthworks Cacao Powder Organic, 1lb

Recommending:

          asin                                              title
0   B001EQ5NMY         Spam with Cheese, 12 Ounce Can (Pack of 6)
1   B0042LH7TK  Green Mountain Coffee Fair Trade Pumpkin Spice...
2   B000BXSRT2     World Confections Candy Cigarettes, Pack of 24
3   B005U4RFUY  Snyders | Old Fashioned Pretzel Sticks 3 pound...
4   B003CK0VVI  Stash Tea Superfruits Sampler, Six Flavor Vari...
5   B00DM07VHA  Biltong Seasoned Dried Beef, Original Sliced, ...
6   B0042RBHDG                  Starbucks Hot Cocoa Powder, 2.5lb
7   B00826N3C6  CBTL Creme Brulee Coffee Capsules By The Coffe...
8   B00A66S32U  Amoretti Premium Syrup, Madagascar Bourbon Van...
9   B00394JNAW  Antimo Caputo Italian &quot;00&quot; Farina Fl...
10  B00

# Cross-Analysis for Cold-Start Users (<= 2 Purchased Items)

In [24]:
cold_start_users = utilities.generate_cold_start_users(train)

In [25]:
for n in tuple(zip([10, 25, 30, 45], [top_ns_10, top_ns_25, top_ns_30, top_ns_45])):
    cold_start_top_ns = dict(filter(lambda x: x[0] in cold_start_users, n[1].items()))
    # evaluate how well the recommended items predicted the future purchases
    # on cold start users
    n_recommended_items = (evaluate_model.
                           evaluate_recommendations(model_name = 'UB-CF',
                                                    top_ns = cold_start_top_ns,
                                                    user_rating_history = test_user_history, 
                                                    item_popularity = item_popularity, 
                                                    n = n[0],
                                                    mf_based = False))

The UB-CF has an average recall@10: 0.02074, average novelty@10: 0.95150
The UB-CF has an average recall@25: 0.02816, average novelty@25: 0.97078
The UB-CF has an average recall@30: 0.02970, average novelty@30: 0.97368
The UB-CF has an average recall@45: 0.03719, average novelty@45: 0.97812


# Generating Recommended Items DataFrame

In [39]:
def generate_recommendations_df(
    train: pd.DataFrame,
    n_recommendations: dict,
    algo_name: str,
    mf_based=False,
    max_recommended: int = 45,
):
    """ """
    
    try:
        top_ns = n_recommendations[max_recommended][0]
        top_ns_df = pd.DataFrame.from_dict(top_ns).T.reset_index()
        # transform column data to row wise
        top_ns_df = top_ns_df.melt(
            id_vars="index", var_name="item_rank", value_name="recommended_items"
        ).sort_values(by=["index", "item_rank"])
    except ValueError:
        top_ns = n_recommendations[max_recommended][0]
        top_ns_df = (pd.DataFrame
                     .from_dict(n_recommendations[45][0], orient="index")
                     .stack()
                     .to_frame().
                     reset_index())
    
    # rename columns
    top_ns_df.columns = ["reviewerID", "item_rank", "asin"]    
    # add the in columns specifying top-n and algorithm
    top_ns_df["algorithm"] = algo_name 

    if mf_based:
        top_ns_df["asin"] = top_ns_df["asin"].apply(lambda x: x[0])    

    # add in item title
    item_info = train[["asin", "title"]].drop_duplicates()    
    recommendation_df = top_ns_df.merge(item_info, how="left", on="asin")

    return recommendation_df

In [47]:
max_recommendations = generate_recommendations_df(train, n_recommendations, "UB-CF", mf_based=False)

# Store in `SQLite` DB

In [43]:
engine = create_engine("sqlite:///recommender.db", echo=True)

In [46]:
max_recommendations.to_sql(f"{CATEGORY}", con=engine, if_exists="append")

2021-09-22 23:15:04,103 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Grocery_and_Gourmet_Food")
2021-09-22 23:15:04,104 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-09-22 23:15:04,258 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-09-22 23:15:07,311 INFO sqlalchemy.engine.Engine INSERT INTO "Grocery_and_Gourmet_Food" ("index", "reviewerID", item_rank, asin, algorithm, title) VALUES (?, ?, ?, ?, ?, ?)
2021-09-22 23:15:07,311 INFO sqlalchemy.engine.Engine [generated in 2.58914s] ((0, 'A00177463W0XWB16A9O05', 0, 'B0029XDZKI', 'UB-CF', "Keurig, Gloria Jean's, Hazelnut, K-Cup Counts, 50 Count"), (1, 'A00177463W0XWB16A9O05', 1, 'B001CHFUDC', 'UB-CF', 'Coffee People Donut Shop Regular Medium Roast Extra Bold, K-Cup Portion Pack for Keurig K-Cup Brewers 24-Count (Pack of 2)'), (2, 'A00177463W0XWB16A9O05', 2, 'B000NERTSE', 'UB-CF', 'Hormel Premium Real Crumbled Bacon 20 oz'), (3, 'A00177463W0XWB16A9O05', 3, 'B005U4RFUY', 'UB-CF', 'Snyders | Old Fashioned Pretzel Sticks 3 pounds